# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [14]:
import pandas as pd
reviews = pd.read_csv("reviews_sample.csv")
reviews.rename(columns={"Unnamed: 0" : "Индексы"}, inplace=True)
recipes = pd.read_csv("recipes_sample.csv", usecols=["id","name","minutes","submitted","description","n_ingredients"])

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [15]:
import xlwings as xw
wb = xw.Book()

sht = wb.sheets["Лист1"]
sht.name = "Отзывы"
sht.range("A1").value = reviews.sample(frac = 0.05)

sht = wb.sheets.add(name="Рецепты")
sht = wb.sheets["Рецепты"]
sht.range("A1").value = recipes.sample(frac = 0.05)

wb.save("recipes.xlsx")
wb.close()

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [16]:
wb = xw.Book("recipes.xlsx")
sht = wb.sheets("Рецепты")

last_row = sht.range('D1').end('down').row
mins = sht.range('D1:D' + str(last_row))[1:].value

secs = [int(i)*60 if i is not None else None for i in mins]
sht.range("H1").value = "Секунды"
sht.range("H2").options(transpose=True).value = secs

wb.save()
wb.close()

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [17]:
wb = xw.Book("recipes.xlsx")
sht = wb.sheets("Рецепты")

last_row = sht.range('D1').end('down').row
fml = sht.range("I2").formula = f"=D2*60"
xw.Range("I2:I"+str(last_row)).formula = fml
sht.range("I1").value = "seconds_formula"

wb.save()
wb.close()

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [18]:
wb = xw.Book("recipes.xlsx")
sht = wb.sheets("Рецепты")

column_range = sht.range("H1:I1")
column_range.font.bold = True
column_range.autofit()

wb.save()
wb.close()

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [19]:
wb = xw.Book("recipes.xlsx")
sht = wb.sheets("Рецепты")

for i in sht["D2"].expand("down"):
    if i.value < 5:
        i.color = (0, 255, 0)
    elif i.value > 10:
        i.color = (255,0,0)
    else:
        i.color = (255,255,0)

wb.save()
wb.close()

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [22]:
# Подключение к файлу Excel
wb = xw.Book("recipes.xlsx")

# Получение доступа к листам "Рецепты" и "Отзывы"
recipes_sheet = wb.sheets["Рецепты"]
reviews_sheet = wb.sheets["Отзывы"]

# Определение диапазона столбца "C" на листе "Рецепты" от ячейки C2 до последней заполненной
last_row_recipes = recipes_sheet.range("C" + str(recipes_sheet.cells.last_cell.row)).end("up").row

# Добавление столбца "n_reviews" на лист "Рецепты"
recipes_sheet.range("J1").value = "n_reviews"  # Заголовок столбца

# Заполнение столбца "n_reviews" с использованием формулы COUNTIF
for i in range(2, last_row_recipes + 1):
    recipe_id = recipes_sheet.range(f"C{i}").value
    formula = f'=COUNTIF(Отзывы!$D:$D, {recipe_id})'
    recipes_sheet.range(f"J{i}").formula = formula
wb.save()

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 